In [ ]:
import as_seg.scripts.overall_scripts as scr
import as_seg.data_manipulation as dm
from as_seg.model.current_plot import *

import numpy as np
import math

# RWC

In [1]:
song_range = range(1,101)
subdivision = 96
hop_length = 32
hop_length_seconds = hop_length/44100
scores = math.inf * np.ones((len(song_range),2,3))
for idx_song, song in enumerate(song_range):
    bars, references_segments = scr.load_bar_annot_song_RWC(song, hop_length)

    aligned_ref = dm.align_segments_on_bars(references_segments, bars)
    
    prec05, rap05, f_mes05 = dm.compute_score_of_segmentation(references_segments, aligned_ref, window_length = 0.5)
    scores[idx_song, 0] = [round(prec05,4),round(rap05,4),round(f_mes05,4)]
    prec3, rap3, f_mes3 = dm.compute_score_of_segmentation(references_segments, aligned_ref, window_length = 3)
    scores[idx_song, 1] = [round(prec3,4),round(rap3,4),round(f_mes3,4)]

print([np.mean(scores[:,0,i]) for i in range(3)])
print([np.mean(scores[:,1,i]) for i in range(3)])

[0.964631, 0.9620820000000001, 0.963322]
[1.0, 0.997298, 0.9986119999999999]


# SALAMI

In [2]:
import as_seg.scripts.default_path as paths
import as_seg.scripts.overall_scripts as scr
import as_seg.data_manipulation as dm

import math
import numpy as np
import mirdata
import os

salami = mirdata.initialize('salami', data_home = paths.path_entire_salami)
len_salami = len(salami.track_ids)
scores_1 = math.inf * np.ones((len_salami,2,3))
song_idx_1 = 0
scores_2 = math.inf * np.ones((len_salami,2,3))
song_idx_2 = 0

all_tracks = salami.load_tracks()    
file_mirex = open(f"{os.getcwd()}/test_set_salami_gs.txt")

test_dataset = []
for part in file_mirex.readlines():
    line_broken = part.split("\n")
    test_dataset.append(int(line_broken[0]))

for key, track in all_tracks.items():
    if int(key) in test_dataset: # Every file in the test dataset
        try:
            bars = scr.load_or_save_bars(paths.path_data_persisted_salami, track.audio_path)
            try:
                references_segments = salami.load_sections(track.sections_annotator1_uppercase_path).intervals
                aligned_ref = dm.align_segments_on_bars(references_segments, bars)
                prec05, rap05, f_mes05 = dm.compute_score_of_segmentation(references_segments, aligned_ref, window_length = 0.5)
                scores_1[song_idx_1, 0] = [round(prec05,4),round(rap05,4),round(f_mes05,4)]
                prec3, rap3, f_mes3 = dm.compute_score_of_segmentation(references_segments, aligned_ref, window_length = 3)
                scores_1[song_idx_1, 1] = [round(prec3,4),round(rap3,4),round(f_mes3,4)]
                song_idx_1 += 1
            except (TypeError, AttributeError):
                pass

            try:
                references_segments = salami.load_sections(track.sections_annotator2_uppercase_path).intervals
                aligned_ref = dm.align_segments_on_bars(references_segments, bars)

                prec05, rap05, f_mes05 = dm.compute_score_of_segmentation(references_segments, aligned_ref, window_length = 0.5)
                scores_2[song_idx_2, 0] = [round(prec05,4),round(rap05,4),round(f_mes05,4)]
                prec3, rap3, f_mes3 = dm.compute_score_of_segmentation(references_segments, aligned_ref, window_length = 3)
                scores_2[song_idx_2, 1] = [round(prec3,4),round(rap3,4),round(f_mes3,4)]
                song_idx_2 += 1
            except (TypeError, AttributeError):
                pass
        except FileNotFoundError:
            print(f"{key} not found, normal ?")
            
print([np.mean(scores_1[:song_idx_1,0,i]) for i in range(3)])
print([np.mean(scores_1[:song_idx_1,1,i]) for i in range(3)])

print([np.mean(scores_2[:song_idx_2,0,i]) for i in range(3)])
print([np.mean(scores_2[:song_idx_2,1,i]) for i in range(3)])


70 not found, normal ?
[0.8288528925619835, 0.8285099173553719, 0.8286756198347107]
[0.9994677685950413, 0.9990867768595041, 0.9992706611570248]
[0.8193873853211008, 0.8188596330275228, 0.8191137614678898]
[0.9996004587155963, 0.998784633027523, 0.9991766055045872]
